# My Playground for Neural Network

In [549]:
from utils import *
from stratified import *
from run import *

In [550]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory
def importwinedata():
    wine = importfile('hw3_wine.csv', '\t')
    winecategory = {}
    for i in wine[0]:
        winecategory[i] = 'numerical'
    winecategory["# class"] = 'class'
    winedata = np.array(wine[1:]).astype(float)
    return winedata, winecategory
def importcmcdata():
    cmc = importfile('cmc.data', ',')
    cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
    "Husband's education":"categorical","Number of children ever born":"numerical",
    "Wife's religion":"binary","Wife's now working?":"binary",
    "Husband's occupation":"categorical","Standard-of-living index":"categorical",
    "Media exposure":"binary","Contraceptive method used":"class"}
    cmcdata = np.array(cmc).astype(int)
    return cmcdata, cmccategory
def importcancerdata():
    cancer = importfile('hw3_cancer.csv', '\t')
    cancercategory = {}
    for i in cancer[0]:
        cancercategory[i] = 'numerical'
    cancercategory["Class"] = 'class'
    cancerdata = np.array(cancer[1:]).astype(float)
    return cancerdata, cancercategory

housedata, housecategory = importhousedata()
winedata, winecategory = importwinedata()
cmcdata,cmccategory = importcmcdata()
cancerdata, cancercategory = importcancerdata()
example_category = {'ece': 'numerical','cs': 'categorical','phy': 'categorical','math': 'categorical','musi': 'categorical','grade': 'class'}
#example_data = np.array([[100,0,1,1,0,0],[123,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2]])
example_data = np.array([[10,0,1,1,0,0],[23,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2],[122,0,1,1,0,0],[124,1,0,1,1,1],[44,1,0,0,0,2],[40,1,0,1,0,3],[73,0,1,0,0,2]])

In [551]:
ohe_house_data, ohe_house_category = onehotencoder(housedata, housecategory)
ohe_wine_data, ohe_wine_category = onehotencoder(winedata,winecategory)
ohe_cmc_data, ohe_cmc_category = onehotencoder(cmcdata,cmccategory)
ohe_example_data, ohe_example_category = onehotencoder(example_data,example_category)
ohe_cancer_data, ohe_cancer_category = onehotencoder(cancerdata,cancercategory)

In [552]:
def g(x): # sigmoid function
    return np.exp(x) / (1 + np.exp(x))

In [553]:
def initialize_weights(ohe_category,layer_parameter, biasterm=True):
    weight_matrix_list = []

    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category  
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm == True else 0
    
    updatedlayerparameterwbias = [len(inputcategory)+b] + list(np.array(layer_parameter)+b) + [len(outputcategory)] # [inputlayer, layerparameters, outputlayer]
    for i in range(len(updatedlayerparameterwbias)-1):
        layernow = updatedlayerparameterwbias[i]
        layernext = updatedlayerparameterwbias[i+1]-1 if i !=len(updatedlayerparameterwbias)-2 else updatedlayerparameterwbias[i+1] 
        # ^ for the last layer, the bias is not included, so don't need to minus 1 ^
        init_weight = np.random.rand(layernext,layernow) * 2 - 1 # initialize the weight with random number between -1 and 1
        weight_matrix_list.append(init_weight)
        
    return weight_matrix_list

In [554]:
# Forward propagation vectorized
def forward_network(normed_hotted_data,ohe_category,weights_list,biasterm=True, minibatchk = 15):
    normed_ohe_copy = normed_hotted_data.copy()
    np.random.shuffle(normed_ohe_copy)
    splitted = np.array_split(normed_ohe_copy, minibatchk)
    # random_indices = np.random.choice(435, size=k, replace=False)
    # random_batch = normed_hotted_data[random_indices].T
    
    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm else 0
    
    for onebatch in splitted:
        onebatch = onebatch.T
        input_data = onebatch[inputindex].mean(axis=1)
        output_data = onebatch[outputindex].mean(axis=1)
        input_data = np.append(1,input_data) if b == 1 else input_data

        current_layer_a = input_data # input layer is the current layer
        current_layer_index = 0
        for theta in weights_list:
            z = np.dot(theta,current_layer_a)
            a = g(z)
            current_layer_a = np.append(1,a) if (b == 1) and (current_layer_index+1 != len(weights_list)) else a
            current_layer_index += 1
        
        output_predict = current_layer_a # the last attribute is the output for this batch.
        print(output_predict,'prediction')
        print(output_data,'real')
    # random_batch_input = random_batch[inputindex].mean(axis=1)

    
    a_lists = [] # list of np.array matrix, attribute of each layer
    for i in range(len(weights_list)):
        pass

    return input_data

In [555]:
w = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
indice = [0,1,3]
w = np.array(w)
z = w[indice]
print(z)
input = z.mean(axis=1)
print(input)
input_data = np.append(1,input)
print(input_data)
haihaihai = np.random.rand(5,3) * 2 - 1
print(haihaihai)
nextz = np.dot(haihaihai,input)
print(nextz)

[[ 1  2  3]
 [ 4  5  6]
 [10 11 12]]
[ 2.  5. 11.]
[ 1.  2.  5. 11.]
[[-0.27062698  0.88493701  0.55635652]
 [ 0.01638038 -0.50418971 -0.73809031]
 [ 0.9601239  -0.38120284  0.2244981 ]
 [-0.50456158  0.96803643 -0.6060317 ]
 [ 0.78142564  0.99237342  0.06301358]]
[ 10.00335285 -10.60718121   2.48371266  -2.83528972   7.21786775]


In [556]:
layerdataparameter = [4,5,4] # [First layer, Second layer, Third layer], not including the bias and input & output layer
weights = initialize_weights(ohe_house_category,layerdataparameter)
print([len(i) for i in weights])
kk = forward_network(ohe_house_data,ohe_house_category,weights)
print(kk)

[4, 5, 4, 2]
[0.54366048 0.5045535 ] prediction
[0.5862069 0.4137931] real
[0.54542072 0.50507166] prediction
[0.48275862 0.51724138] real
[0.54692997 0.50450692] prediction
[0.51724138 0.48275862] real
[0.54308138 0.50560605] prediction
[0.51724138 0.48275862] real
[0.54376167 0.50358622] prediction
[0.65517241 0.34482759] real
[0.54300771 0.50387178] prediction
[0.79310345 0.20689655] real
[0.54519611 0.50630401] prediction
[0.68965517 0.31034483] real
[0.54461631 0.50455432] prediction
[0.72413793 0.27586207] real
[0.54237176 0.50546683] prediction
[0.65517241 0.34482759] real
[0.54430727 0.50610746] prediction
[0.55172414 0.44827586] real
[0.54196217 0.5062817 ] prediction
[0.65517241 0.34482759] real
[0.54476617 0.50503369] prediction
[0.55172414 0.44827586] real
[0.54281787 0.50591947] prediction
[0.65517241 0.34482759] real
[0.54415555 0.50600066] prediction
[0.65517241 0.34482759] real
[0.54591066 0.50380454] prediction
[0.51724138 0.48275862] real
[1.         0.03448276 0.7586

In [557]:
k = 5
wok = np.array([[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14],[15,16]])
whaha = len(wok)
print(whaha)
print(wok.shape[0])
random_indices = np.random.choice(whaha, size=k, replace=False)
random_rows = wok[random_indices]
print(random_rows)

8
8
[[ 1  2]
 [ 3  4]
 [ 7  8]
 [13 14]
 [ 5  6]]


In [558]:
layernow = 3
layernext = 2
init_weight = np.random.rand(layernext,layernow) * 2 - 1
a_now = np.random.rand(layernow,1)
print(init_weight)
print(a_now)
c = np.dot(init_weight,a_now)
print(c)

[[-0.46399656 -0.04384985  0.99950012]
 [-0.11994951 -0.27985821 -0.74578831]]
[[0.76513216]
 [0.02872369]
 [0.28336969]]
[[-0.07305018]
 [-0.3111496 ]]
